<!DOCTYPE html>
<html>
<head>
<title> <h1><center><span class="header-section-number"> 10. </span> Tensorflow 2.0. Regresión Lineal</h1></center></title>
</head>
   https://colab.research.google.com/drive/1F3DIXAv-5vopS3ffBVp2vKgXd5u7lcO9
<body>
</body>
</html>

<h2><span class="header-section-number"> 10.1 </span> Introducción </h2>

En esta práctica implementamos el método de regresión lineal en Tensorflow 2.0. Usaremos el conjunto de datos *Boston housing* y el estimator  **LinearRegressor**.

<h2><span class="header-section-number"> 10.2 </span>Importar módulos requeridos </h2>


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import seaborn as sb
import tensorflow as tf
from tensorflow import keras as ks

from tensorflow.estimator import LinearRegressor
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing

print(tf.__version__)

2.0.0


<h2><span class="header-section-number"> 10.3 </span> Cargar y configurar la base de datos Boston housing </h2>

In [2]:
boston_load    = datasets.load_boston()
feature_columns = boston_load.feature_names
target_column  = boston_load.target

boston_data = pd.DataFrame(boston_load.data,columns=feature_columns).astype(np.float32)
boston_data['MEDV'] = target_column.astype(np.float32)
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.199997,4.0900,1.0,296.0,15.300000,396.899994,4.98,24.000000
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.900002,4.9671,2.0,242.0,17.799999,396.899994,9.14,21.600000
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.099998,4.9671,2.0,242.0,17.799999,392.829987,4.03,34.700001
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.799999,6.0622,3.0,222.0,18.700001,394.630005,2.94,33.400002
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.200001,6.0622,3.0,222.0,18.700001,396.899994,5.33,36.200001


In [3]:
boston_data.shape

(506, 14)

<h2><span class="header-section-number"> 10.4 </span> Primer acercamiento gráfico a los datos   </h2>

Revisaremos las relaciones entre variables, usando pairplot y un gráfico de correlación

In [ ]:
sb.pairplot(boston_data, diag_kind ="kde")

In [46]:
correlation_data = boston_data.corr()
correlation_data.style.background_gradient(cmap='coolwarm', axis =None)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
CRIM,1,-0.200469,0.406583,-0.0558916,0.420972,-0.219247,0.352734,-0.37967,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
ZN,-0.200469,1,-0.533828,-0.0426967,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.17552,-0.412995,0.360445
INDUS,0.406583,-0.533828,1,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.72076,0.383248,-0.356977,0.6038,-0.483725
CHAS,-0.0558916,-0.0426967,0.062938,1,0.0912028,0.0912512,0.0865178,-0.0991758,-0.00736824,-0.0355865,-0.121515,0.0487885,-0.0539293,0.17526
NOX,0.420972,-0.516604,0.763651,0.0912028,1,-0.302188,0.73147,-0.76923,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
RM,-0.219247,0.311991,-0.391676,0.0912512,-0.302188,1,-0.240265,0.205246,-0.209847,-0.292048,-0.355502,0.128069,-0.613808,0.69536
AGE,0.352734,-0.569537,0.644779,0.0865178,0.73147,-0.240265,1,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
DIS,-0.37967,0.664408,-0.708027,-0.0991758,-0.76923,0.205246,-0.747881,1,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
RAD,0.625505,-0.311948,0.595129,-0.00736824,0.611441,-0.209847,0.456022,-0.494588,1,0.910228,0.464741,-0.444413,0.488676,-0.381626
TAX,0.582764,-0.314563,0.72076,-0.0355865,0.668023,-0.292048,0.506456,-0.534432,0.910228,1,0.460853,-0.441808,0.543993,-0.468536


<h2><span class="header-section-number"> 10.4 </span> Estadísticas Descriptivas  </h2>

Tndencia central y dispersión

In [47]:
stats = boston_data.describe()
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
CRIM,506.0,3.613524,8.601545,0.00632,0.082045,0.256510,3.677083,88.976196
ZN,506.0,11.363636,23.322453,0.00000,0.000000,0.000000,12.500000,100.000000
INDUS,506.0,11.136780,6.860353,0.46000,5.190000,9.690000,18.100000,27.740000
CHAS,506.0,0.069170,0.253994,0.00000,0.000000,0.000000,0.000000,1.000000
NOX,506.0,0.554695,0.115878,0.38500,0.449000,0.538000,0.624000,0.871000
RM,506.0,6.284634,0.702617,3.56100,5.885500,6.208500,6.623500,8.780000
AGE,506.0,68.574898,28.148861,2.90000,45.025000,77.500000,94.074999,100.000000
DIS,506.0,3.795043,2.105710,1.12960,2.100175,3.207450,5.188425,12.126500
RAD,506.0,9.549407,8.707259,1.00000,4.000000,5.000000,24.000000,24.000000
TAX,506.0,408.237152,168.537109,187.00000,279.000000,330.000000,666.000000,711.000000


<h2><span class="header-section-number"> 10.5 </span> Prepara datos para entrenamiento  </h2>


In [5]:
x_data = boston_data.iloc[:, 0:(boston_data.shape[1]-1)]
y_data = boston_data.iloc[:,-1]

In [7]:
print(x_data.shape, y_data.shape)

(506, 13) (506,)


<h2><span class="header-section-number"> 10.6 </span> Divide los datos  </h2>

Selecciona 20% de los datos para validación y 80% para entrenamiento

In [13]:
training_features, test_features, training_labels, test_labels = train_test_split(x_data,y_data, 
                                                                                  test_size = 0.2)
print('Número de muestras de entrenamiento: ', training_features.shape[0])
print('Número de variables de entrenamiento: ', training_features.shape[1])
print('Número de etiquetas de validación: ', test_labels.shape[0])


Número de muestras de entrenamiento:  404
Número de variables de entrenamiento:  13
Número de etiquetas de validación:  102


<h2><span class="header-section-number"> 10.7 </span> Normaliza los datos  </h2>

Usaremos la herramienta de preprocesamiento StandardScaler.

Se crea una instancia de StandardScaler, usando los datos de entranamiento (training features). 

Con esta instancia se normalizan los datos de entrenamiento y los de validación (testing features)

In [14]:
# crea una instancia de StandardScaler con los pa
scaler = preprocessing.StandardScaler().fit(training_features)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
scaler.mean_

array([3.44749051e+00, 1.15420792e+01, 1.11471783e+01, 6.93069307e-02,
       5.55925000e-01, 6.30787376e+00, 6.82245050e+01, 3.80189085e+00,
       9.53960396e+00, 4.09123762e+02, 1.84049507e+01, 3.56258785e+02,
       1.26374010e+01])

In [16]:
scaler.scale_

array([7.88501379e+00, 2.33733046e+01, 6.77154395e+00, 2.53975355e-01,
       1.17007126e-01, 7.14404252e-01, 2.85742501e+01, 2.10129606e+00,
       8.63110391e+00, 1.64498446e+02, 2.20719492e+00, 9.22535522e+01,
       7.22547867e+00])

In [36]:
# scale trainig and test features
normed_training_features = pd.DataFrame(scaler.transform(training_features),columns=training_features.columns.values)
normed_test_features     = pd.DataFrame(scaler.transform(test_features),columns=test_features.columns.values)

stats = (normed_training_features.describe()).transpose()
stats

,count,mean,std,min,25%,50%,75%,max
CRIM,404.0,2.950725e-09,1.00124,-0.436419,-0.426589,-0.400496,0.018698,8.888585
ZN,404.0,4.131015e-08,1.00124,-0.493815,-0.493815,-0.493815,0.094463,3.784571
INDUS,404.0,1.180290e-09,1.00124,-1.578248,-0.888598,-0.215191,1.026771,2.450375
CHAS,404.0,7.081740e-09,1.00124,-0.272888,-0.272888,-0.272888,-0.272888,3.664502
NOX,404.0,2.360580e-09,1.00124,-1.460809,-0.915970,-0.153196,0.581802,2.692785
RM,404.0,-1.770435e-09,1.00124,-3.422256,-0.586676,-0.126502,0.457901,3.460402
AGE,404.0,-1.180290e-09,1.00124,-2.286132,-0.810153,0.305362,0.913427,1.112033
DIS,404.0,4.721160e-09,1.00124,-1.271735,-0.820822,-0.301905,0.672304,3.961655
RAD,404.0,-2.006493e-08,1.00124,-0.989399,-0.641819,-0.525959,1.675382,1.675382
TAX,404.0,2.655652e-08,1.00124,-1.350309,-0.760638,-0.438446,1.561573,1.835131


<h2><span class="header-section-number"> 10.8 </span> Prepara los datos para Tensorflow 2.0 </h2>

Construye una tuberia (pipeline) para prepar los datos para Tensorflow 2.0


In [37]:
# define the pipeline
def feed_input(features_dataframe, target_dataframe,
              num_of_epochs=10, shuffle=True, batch_size = 32):
    def input_feed_function():
        dataset = tf.data.Dataset.from_tensor_slices((dict(features_dataframe),target_dataframe))
        if shuffle:
            dataset = dataset.shuffle(2000)
        dataset = dataset.batch(batch_size).repeat(num_of_epochs)
        return dataset
    return input_feed_function


In [50]:
# use the pipeline to prepare the data to Tensorflow 2.0
train_feed_input = feed_input(normed_training_features, training_labels)
train_feed_input_testing = feed_input(normed_training_features, training_labels,num_of_epochs=1,shuffle=False)
test_feed_input = feed_input(normed_test_features,test_labels,num_of_epochs=1, shuffle=False)

<h2><span class="header-section-number"> 10.9 </span> Entrenamiento del modelo en Tensorflow 2.0 </h2>


In [51]:
feature_columns_numeric =[tf.feature_column.numeric_column(m) 
                          for m in training_features.columns]
Linear_model = LinearRegressor(feature_columns = feature_columns_numeric,
                              optimizer ='RMSProp')
Linear_model.train(train_feed_input)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp1p05miw5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd2a43c8650>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done c

<h2><span class="header-section-number"> 10.10 </span> Predicciones </h2>


In [52]:
train_predictions = Linear_model.predict(train_feed_input_testing)
test_predictions  = Linear_model.predict(test_feed_input)

In [53]:
train_predictions_series = pd.Series(p['predictions'][0] for p in train_predictions)
test_predictions_series  = pd.Series(p['predictions'][0] for p in test_predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1p05miw5/model.ckpt-130
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1p05miw5/model.ckpt-130
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [54]:
train_predictions_df =pd.DataFrame(train_predictions_series,columns= ['predictions'])
test_predictions_df =pd.DataFrame(test_predictions_series,columns= ['predictions'])

In [55]:
training_labels.reset_index(drop=True, inplace=True)
train_predictions_df.reset_index(drop=True, inplace=True)

In [44]:
test_labels.reset_index(drop=True, inplace=True)
test_predictions_df.reset_index(drop=True, inplace=True)

In [56]:
train_labels_with_predictors = pd.concat([training_labels,train_predictions_df],axis=1)
test_labels_with_predictors = pd.concat([test_labels,test_predictions_df],axis=1)

<h2><span class="header-section-number"> 10.11 </span> Validación </h2>


In [57]:
def calculate_errors_and_r2(y_true, y_pred):
    mean_squared_err = (mean_squared_error(y_true, y_pred))
    root_mean_squared_err = np.sqrt(mean_squared_err)
    r2 = r2_score(y_true,y_pred)
    return mean_squared_err, root_mean_squared_err, r2


In [58]:
tr_mse, tr_rmse, tr_r2 =  calculate_errors_and_r2(training_labels, train_predictions_series)

In [59]:
test_mse, test_rmse, test_r2 =  calculate_errors_and_r2(test_labels, test_predictions_series)

In [60]:
print('train_mse= ', tr_mse)
print('train_rmse= ', tr_rmse)
print('train_r2= ', tr_r2)
print('test_mse= ', test_mse)
print('test_rmse= ', test_rmse)
print('test_r2= ', test_r2)

train_mse=  26.127304685283242
train_rmse=  5.111487521777124
train_r2=  0.7100304003096852
test_mse=  15.963933322772794
test_rmse=  3.9954891218438817
test_r2=  0.7351805028571958


In [61]:
feature_columns_numeric

[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PTRATIO'

In [62]:
type(feature_columns_numeric)

list